## **Introduction**

This notebook demonstrates how to model symbolic memory using the Z3 theorem prover, a powerful SMT (Satisfiability Modulo Theories) solver.  

It simulates symbolic execution over memory using symbolic pointers and values. The core functionalities involve:

1. **Symbolic Memory Modeling**:  
   Using a map to represent memory, where each address can store a Z3 expression. This allows us to model behavior like `malloc`, `store`, and `load`.

2. **Z3 Integration**:  
   Using the Z3 Python API to model constraints, evaluate expressions, and interact with a symbolic memory system.

---

### **Z3Mgr Class**

The `Z3Mgr` class is the core component that wraps interactions with Z3 and maintains the symbolic memory.  

It provides the following functionality:

- Declaring symbolic integers.
- Storing symbolic values into symbolic addresses.
- Loading values from memory addresses.
- Evaluating and printing symbolic expressions.

---

---

### **Your Tasks**

This notebook contains placeholders for implementing Z3 code in Python to model and verify the behavior of the provided C code examples. For each test (from 1 to 10), the corresponding C code has already been written(I've already given you 1 example with test0). Your task is to:

- Translate the logic of the C code into Z3 Python code.
- Use the `Z3Mgr` class to model symbolic memory, pointers, and values.
- Implement the required Z3 logic to verify the assertions in the C code.
- Ensure that the Z3 code correctly models the behavior of the C code and passes the provided tests.

For each test, you should:

1. **Model the C Code**:  
   Write Z3 Python code to represent the logic of the C code, including memory allocation, pointer manipulation, and value assignments.

2. **Verify Assertions**:  
   Use Z3 to verify that the assertions in the C code hold true. This involves creating constraints and checking their satisfiability.

3. **Complete the Test Functions**:  
   Implement the missing Z3 logic in the test functions (`test1` to `test10`) to ensure they pass successfully.

---


Let's begin by defining the `Z3Mgr` class.


In [1]:
from typing import Union

import z3
import sys


class Z3Mgr:
    def __init__(self, num_of_map_elems: int) -> None:
        assert isinstance(num_of_map_elems,
                          int), "num_of_map_elems is not a valid integer, the type of num_of_map_elems is {}".format(
            type(num_of_map_elems))
        self.ctx = z3.Context()
        self.solver = z3.Solver(ctx=self.ctx)
        self.var_id_to_expr_map = {}
        self.max_num_of_expr = num_of_map_elems
        self.str_to_id_map = {}
        self.current_expr_idx = 0

        self.addressMark = 0x7f000000

        index_sort = z3.IntSort(self.ctx)
        element_sort = z3.IntSort(self.ctx)
        array_sort = z3.ArraySort(index_sort, element_sort)
        self.loc_to_val_map = z3.Const('loc2ValMap', array_sort)

    def storeValue(self, loc: Union[z3.ExprRef,int], value:Union[z3.ExprRef, int]) -> z3.ExprRef:
        assert isinstance(loc, (z3.ExprRef,int)), f"loc is not a valid z3 expression or int, got {type(loc)}"
        assert isinstance(value, (z3.ExprRef, int)), f"value is not a valid z3 expression or int, got {type(value)}"

        deref = self.getEvalExpr(loc)
        assert self.isVirtualMemAddress(deref.as_long()), "Pointer operand is not a physical address"
        self.loc_to_val_map = z3.Store(self.loc_to_val_map, deref, value)
        return self.loc_to_val_map

    def loadValue(self, loc: Union[z3.ExprRef,int]) -> z3.ExprRef:
        assert isinstance(loc, (z3.ExprRef,int)), "loc is not a valid z3 expression or int, the type of loc is {}".format(type(loc))
        deref = self.getEvalExpr(loc)
        assert self.isVirtualMemAddress(deref.as_long()), "Pointer operand is not a physical address"
        return z3.Select(self.loc_to_val_map, deref)

    def getEvalExpr(self, e: Union[z3.ExprRef, int]) -> z3.ExprRef:
        assert isinstance(e, (z3.ExprRef,int)), "e is not a valid z3 expression or int, the type of e is {}".format(type(e))
        if isinstance(e, int):
            e = z3.IntVal(e, self.ctx)
            return e
        else:
            res = self.solver.check()
            assert res != z3.unsat, "unsatisfied constraints! Check your contradictory constraints added to the solver"
            model = self.solver.model()
            val = model.eval(e, model_completion=True)
            return val

    def hasZ3Expr(self, expr_name: str) -> bool:
        assert isinstance(expr_name, str), "expr_name is not a valid string, the type of expr_name is {}".format(
            type(expr_name))
        return expr_name in self.str_to_id_map

    def getZ3Expr(self, expr_name: str) -> z3.ExprRef:
        assert isinstance(expr_name, str), "expr_name is not a valid string, the type of expr_name is {}".format(
            type(expr_name))
        if expr_name in self.str_to_id_map:
            return self.var_id_to_expr_map[self.str_to_id_map[expr_name]]
        else:
            self.current_expr_idx += 1
            self.str_to_id_map[expr_name] = self.current_expr_idx
            expr = z3.Int(str(expr_name), ctx=self.ctx)
            self.updateZ3Expr(self.current_expr_idx, expr)
            return expr

    def updateZ3Expr(self, idx: int, target: Union[z3.ExprRef, int]) -> None:
        assert isinstance(idx, int), "idx is not a valid integer, the type of idx is {}".format(type(idx))
        assert isinstance(target, (z3.ExprRef, int)), "target is not a valid z3 expression or int, the type of target is {}".format(type(target))

        if self.max_num_of_expr < idx + 1:
            raise IndexError("idx out of bound for map access, increase map size!")
        self.var_id_to_expr_map[idx] = target

    def getZ3Val(self, val: int) -> z3.ExprRef:
        assert isinstance(val, int), "val is not a valid integer, the type of val is {}".format(type(val))
        return z3.IntVal(val, self.ctx)

    def isVirtualMemAddress(self, val: int) -> bool:
        assert isinstance(val, int), "val is not a valid integer, the type of val is {}".format(type(val))
        return val > 0 and (val & self.addressMark) == self.addressMark

    def getVirtualMemAddress(self, idx: int) -> int:
        assert isinstance(idx, int), "idx is not a valid integer, the type of idx is {}".format(type(idx))
        return self.addressMark + idx

    def getMemObjAddress(self, expr_name: Union[str,z3.ExprRef]) -> z3.ExprRef:
        assert isinstance(expr_name, (str,z3.ExprRef)), "expr_name is not a valid string or z3 Expression, the type of expr_name is {}".format(
            type(expr_name))
        if isinstance(expr_name, z3.ExprRef):
            expr_name = str(expr_name)
        self.getZ3Expr(expr_name)
        if expr_name in self.str_to_id_map:
            e = self.getZ3Val(self.getVirtualMemAddress(self.str_to_id_map[expr_name])).as_long()
            self.updateZ3Expr(self.str_to_id_map[expr_name], e)
            return e
        else:
            assert False, "Invalid memory object name"

    def getGepObjAddress(self, base_expr: z3.ExprRef, offset: int) -> z3.ExprRef:
        assert isinstance(base_expr,
                          z3.ExprRef), "base_expr is not a valid z3 expression, the type of base_expr is {}".format(
            type(base_expr))
        assert isinstance(offset, int), "offset is not a valid integer, the type of offset is {}".format(type(offset))
        base_obj_name = str(base_expr)
        if base_obj_name in self.str_to_id_map:
            base_obj_id = self.str_to_id_map[base_obj_name]
            gep_obj_id = base_obj_id + offset
            if base_obj_id == gep_obj_id:
                return base_expr
            else:
                gep_obj_id += self.max_num_of_expr // 2
                e = self.getZ3Val(self.getVirtualMemAddress(gep_obj_id)).as_long()
                self.updateZ3Expr(gep_obj_id, e)
                return e
        else:
            assert False, "Invalid base object name"

    def addToSolver(self, expr: z3.ExprRef) -> None:
        assert isinstance(expr, z3.ExprRef), "expr is not a valid z3 expression, the type of expr is {}".format(
            type(expr))
        self.solver.add(expr)

    def resetSolver(self) -> None:
        self.solver.reset()
        self.str_to_id_map = {}
        self.var_id_to_expr_map = {}
        self.current_expr_idx = 0
        self.loc_to_val_map = z3.Const('loc2ValMap', self.loc_to_val_map.sort())

    def printExprValues(self):
        print("-----------Var and Value-----------")
        for nIter, Id in self.str_to_id_map.items():
            e = self.getEvalExpr(self.getZ3Expr(nIter))
            value = e.as_long()
            exprName = f"Var{Id} ({nIter})"
            if self.isVirtualMemAddress(value):
                print(f"{exprName}\t Value: {hex(value)}")
            else:
                print(f"{exprName}\t Value: {value}")
        print("-----------------------------------------")

In [2]:
def checkNegateAssert(z3_mgr, q):
    """
    Check the negation of the assertion.

    Args:
        z3_mgr: Instance of Z3Mgr
        q: Assertion to check

    Returns:
        True if the negation of the assertion is unsatisfiable, False otherwise
    """
    z3_mgr.solver.push()
    z3_mgr.addToSolver(z3.Not(q))
    res = z3_mgr.solver.check() == z3.unsat
    z3_mgr.solver.pop()
    return res

### Z3Mgr Class and check_negate_assert Function

The `Z3Mgr` class and the `check_negate_assert` function must be defined and executed before running any of the test cases. Ensure that these are implemented and executed in your environment.

---

### Test Case: test0

Below is an example of a test case (`test0`). Make sure to run the `Z3Mgr` class and `check_negate_assert` function before executing this test case.
This test case mirrors a simple C program that uses pointers and assertions. The goal is to model the behavior of the C code using Z3 and verify the assertion.
```


In [3]:
'''
    /* A simple example

    int main() {
        int* p;
        int q;
        int* r;
        int x;
    
        p = malloc();
        q = 5;
        *p = q;
        x = *p;
        assert(x==5);
    }
*/
'''

def test0():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    p = z3_mgr.getZ3Expr("p")
    q = z3_mgr.getZ3Expr("q")
    x = z3_mgr.getZ3Expr("x")
    z3_mgr.addToSolver(p == z3_mgr.getMemObjAddress("p"))
    z3_mgr.addToSolver(q == 5)
    z3_mgr.storeValue(p, q)
    z3_mgr.addToSolver(x == z3_mgr.loadValue(p))
    z3_mgr.addToSolver(x == 5)
    
    # DO NOT change the following code
    q = z3_mgr.getZ3Expr("x") == z3_mgr.getZ3Val(5)
    result = checkNegateAssert(z3_mgr, q) and z3_mgr.hasZ3Expr("x") and z3_mgr.getEvalExpr(z3_mgr.getZ3Expr("x")).as_long() == 5
    assert result, "Test 0 failed"
    print("Test 0 passed")

test0()


Test 0 passed


## 🚀 Your Challenge Starts Here!

Now that you've seen how symbolic memory modeling works with Z3, it's time to implement and explore more test cases.

Each test case mirrors a corresponding C program, and you'll simulate its behavior using symbolic expressions in Z3. These tests will help solidify your understanding of:

- Symbolic pointers
- Conditional execution
- Memory storage and retrieval
- Constraint solving

We'll follow a structure similar to **Test 0**, where we initialize symbolic variables, store and load from symbolic memory, and verify results using assertions and Z3 solvers.

Let's continue and implement the remaining test cases one by one!


In [4]:
'''
/*
    // Simple integers

    int main() {
        int a;
        int b;
        a = 0;
        b = a + 1;
        assert(b>0);
    }
*/
'''
def test1():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    a = z3_mgr.getZ3Expr('a')
    b = z3_mgr.getZ3Expr('b')
    
    # TODO: implement your code here


    # assert(b > 0); 
    # You are suggested to write your own results checking here

test1()

AssertionError: Test 1 failed

In [5]:
'''
  // One-level pointers

    int main() {
		int* p;
		int q;
		int b;
		p = malloc;
		*p = 0;
		q = *p;
		*p = 3;
		b = *p + 1;
		assert(b > 3);
    }
'''
def test2():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    p = z3_mgr.getZ3Expr('p')
    q = z3_mgr.getZ3Expr('q')
    b = z3_mgr.getZ3Expr('b')
    
    # TODO: implement your code here
    
    
    # assert(b > 3); 
    # You are suggested to write your own results checking here

test2()

AssertionError: Test 2 failed

In [6]:
'''
    // Mutiple-level pointers

    int main() {
        int** p;
        int* q;
        int* r;
        int x;
    
        p = malloc1(..);
        q = malloc2(..);
        *p = q;
        *q = 10;
        r = *p;
        x = *r;
        assert(x==10);
    }
'''
def test3():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    p = z3_mgr.getZ3Expr('p')
    q = z3_mgr.getZ3Expr('q')
    r = z3_mgr.getZ3Expr('r')
    x = z3_mgr.getZ3Expr('x')
    
    # TODO: implement your code here
    
    
    # assert(x==10);
    # You are suggested to write your own results checking here

test3()

AssertionError: Pointer operand is not a physical address

In [7]:
'''
   // Array and pointers

    int main() {
        int* p;
        int* x;
        int* y;
        int a;
        int b;
        p = malloc;
        x = &p[0];
        y = &p[1]
        *x = 10;
        *y = 11;
        a = *x;
        b = *y;
        assert((a + b)>20);
    }
'''
def test4():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    p = z3_mgr.getZ3Expr('p')
    x = z3_mgr.getZ3Expr('x')
    y = z3_mgr.getZ3Expr('y')
    a = z3_mgr.getZ3Expr('a')
    b = z3_mgr.getZ3Expr('b')
    
    # TODO: implement your code here
    
    
    # assert((a + b)>20);
    # You are suggested to write your own results checking here

test4()

AssertionError: Test 4 failed

In [8]:
'''
    // Branches

    int main(int argv) {
        int a;
        int b;
        int b1;
        a = argv + 1;
        b = 5;
        if(a > 10)
            b = a;
        b1 = b;
        assert(b1 >= 5);
    }
'''
def test5():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    a = z3_mgr.getZ3Expr('a')
    b = z3_mgr.getZ3Expr('b')
    b1 = z3_mgr.getZ3Expr('b1')
    
    # TODO: implement your code here
    
    
    # assert(b1 >= 5);
    # You are suggested to write your own results checking here

test5()

AssertionError: Test 5 failed

In [9]:
'''
    int main() {
       int *a = malloc1;
       int *b = malloc2;
       *a = 5;
       *b = 10;
       int *p;
       if (*a < *b) {
           p = a;
       } else {
           p = b;
       }
       assert(*p == 5);
    }
'''
def test6():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    a = z3_mgr.getZ3Expr('a')
    b = z3_mgr.getZ3Expr('b')
    p = z3_mgr.getZ3Expr('p')
    
    # TODO: implement your code here
    
    
    # assert(*p == 5);
    # You are suggested to write your own results checking here

test6()

AssertionError: Pointer operand is not a physical address

In [10]:
'''
    int main() {
        int a = 1, b = 2, c = 3;
        int d;
        if (a > 0) {
            d = b + c;
        }
        else {
            d = b - c;
        }
        assert(d == 5);
    }
'''
def test7():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    a = z3_mgr.getZ3Expr('a')
    b = z3_mgr.getZ3Expr('b')
    c = z3_mgr.getZ3Expr('c')
    d = z3_mgr.getZ3Expr('d')
    
    # TODO: implement your code here
    
    
    # assert(d == 5);
    # You are suggested to write your own results checking here

test7()

AssertionError: Test 7 failed

In [11]:
'''
    int main() {
        int arr[2] = {0, 1};
        int a = 10;
        int *p;
        if (a > 5) {
            p = &arr[0];
        }
        else {
            p = &arr[1];
        }
        assert(*p == 0);
    }

'''
def test8():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    arr = z3_mgr.getZ3Expr('arr')
    a = z3_mgr.getZ3Expr('a')
    p = z3_mgr.getZ3Expr('p')
    
    # TODO: implement your code here
    

    # assert(*p == 0);
    # You are suggested to write your own results checking here

test8()

AssertionError: Pointer operand is not a physical address

In [12]:
'''
    // Struct and pointers

    struct A{ int f0; int* f1;};
    int main() {
       struct A* p;
       int* x;
       int* q;
       int** r;
       int* y;
       int z;
    
       p = malloc1;
       x = malloc2;
       *x = 5;
       q = &(p->f0);
       *q = 10;
       r = &(p->f1);
       *r = x;
       y = *r;
       z = *q + *y;
       assert(z == 15);
    }
'''
def test9():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    A = z3_mgr.getZ3Expr('A')
    p = z3_mgr.getZ3Expr('p')
    x = z3_mgr.getZ3Expr('x')
    q = z3_mgr.getZ3Expr('q')
    r = z3_mgr.getZ3Expr('r')
    y = z3_mgr.getZ3Expr('y')
    z = z3_mgr.getZ3Expr('z')
    
    # TODO: implement your code here
    
    
    # assert(z == 15);
    # You are suggested to write your own results checking here

test9()

AssertionError: Test 9 failed

In [13]:
'''
    int foo(int z) {
        k = z;
        return k;
    }
    int main() {
      int x;
      int y;
      y = foo(2);
      x = foo(3);
      assert(x == 3 && y == 2);
    }
'''
def test10():
    # DO NOT change the following code
    z3_mgr = Z3Mgr(1000)
    x = z3_mgr.getZ3Expr('x')
    y = z3_mgr.getZ3Expr('y')
    z = z3_mgr.getZ3Expr('z')
    k = z3_mgr.getZ3Expr('k')
    
    # TODO: implement your code here
    

    # assert(x == 3 && y == 2);
    # You are suggested to write your own results checking here

test10()

AssertionError: Test 10 failed

## 🎉 Congratulations!

You have successfully completed all the tasks in this notebook.  
Throughout the exercises, you've:

- Modeled symbolic memory using Z3.
- Implemented symbolic `store` and `load` operations.
- Evaluated symbolic expressions under concrete models.
- Translated C test cases into symbolic Z3-based Python tests.

By completing this notebook, you've taken a significant step forward in mastering symbolic execution and constraint solving using Z3.

Keep exploring, and great work! 💪🚀
